In [1]:
import numpy as np
import cv2

from collections import deque

In [2]:
def preprocessFunc(imagesGen,actionsGen,slideSize=4):
    # image should be 150*600*3 nparray
    # action should be String 
    shortTermMemory = deque([],slideSize)
    vertices = np.array([[(430,30),(430,0),(600,0),(600,30)]]) 
    while True:
        try:
            img = next(imagesGen)
            act = next(actionsGen)
        except StopIteration:
            yield None,None,True
            break
        
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # gray scale   (150,600)
        img = cv2.fillPoly(img,vertices,(255))     # override score
        img = cv2.GaussianBlur(img,(5,5),0)
        img = cv2.Canny(img,100,200)
        img = cv2.resize(img,(120,30),interpolation=cv2.INTER_AREA) # (30,120)
        img = img[:,:,None] # (30,120) -> (30,120,1)
        
        shortTermMemory.append(img)
        
        if len(shortTermMemory) == slideSize:
            state  = np.concatenate(shortTermMemory,axis=2) # (30,120,4)
            action = act if act != 'Space' else 'Up'
            yield state,action,False

In [3]:
class Transitions:
    def __init__(self,prevState,prevAction,state,last=False):
        self.s0 = prevState
        self.a0 = prevAction
        self.s1 = state
        self.r1 = 0 if not last else -1

In [4]:
def bulidTransitions(preprocessGen):
    prevState = None
    prevAction = None
    
    trans = []
    for state,action,last in preprocessGen:
        if prevState is not None:     
            t = Transitions(prevState,prevAction,state,last)
            trans.append(t)
        
        prevState = state
        prevAction = action
    
    return trans 

In [5]:
%%time

import pickle
import os

exist = os.listdir(".\\transitions")
for fn in os.listdir(".\\human"):
    print(fn)
    if fn in exist:
        print("{} already exist, skip buliding.\n".format(fn))
        continue
        
    with open(".\\human\\{}".format(fn),'rb') as file:
        imgs,acts = pickle.load(file)
    imgGen = (img for img in imgs)
    actGen = (act for act in acts)
    preprocessGen = preprocessFunc(imgGen,actGen)
    trans = bulidTransitions(preprocessGen)
    
    with open(".\\transitions\\{}".format(fn),'wb') as file:
        pickle.dump(trans,file)
    print("{} bulid finish.\n".format(fn))

human_0000.p
human_0000.p already exist, skip buliding.

human_0001.p
human_0001.p already exist, skip buliding.

human_0002.p
human_0002.p already exist, skip buliding.

human_0003.p
human_0003.p already exist, skip buliding.

human_0004.p
human_0004.p already exist, skip buliding.

human_0005.p
human_0005.p already exist, skip buliding.

human_0006.p
human_0006.p already exist, skip buliding.

human_0007.p
human_0007.p already exist, skip buliding.

human_0008.p
human_0008.p already exist, skip buliding.

human_0009.p
human_0009.p already exist, skip buliding.

human_0010.p
human_0010.p already exist, skip buliding.

human_0011.p
human_0011.p already exist, skip buliding.

human_0012.p
human_0012.p already exist, skip buliding.

human_0013.p
human_0013.p already exist, skip buliding.

human_0014.p
human_0014.p already exist, skip buliding.

human_0015.p
human_0015.p already exist, skip buliding.

human_0016.p
human_0016.p already exist, skip buliding.

human_0017.p
human_0017.p alrea